In [30]:
import pandas as pd
from datasets import load_dataset
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
from nltk.translate.bleu_score import sentence_bleu
import nltk
from rouge import Rouge
import numpy as np
import fasttext
import fasttext.util
ft = fasttext.load_model('wiki.hi/wiki.hi.bin')
import numpy as np
from numpy.linalg import norm
from datasets import Dataset

C:\Users\saira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0')

C:\Users\saira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_dataset = dataset['test']

In [3]:
test_dataset = test_dataset.map(lambda ex: {'reference_summary': ex['highlights']})

In [5]:
test_dataset

Dataset({
    features: ['article', 'highlights', 'id', 'reference_summary'],
    num_rows: 11490
})

In [21]:
output_list = []
for input in test_dataset:
    print(input['article'])
    print('\n****')
    print(input['highlights'])
    print('\n****')
    print(input['reference_summary'])
    break

(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's ceremony, sa

In [4]:
test_dataset = test_dataset.select(list(range(10)))

In [7]:
test_dataset

Dataset({
    features: ['article', 'highlights', 'id', 'reference_summary'],
    num_rows: 10
})

In [5]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

#src_text = test_dataset['article'][0]
output_list = []
model_name = "google/pegasus-cnn_dailymail"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

for input in test_dataset:
    batch = tokenizer(input['article'], truncation=True, padding="longest", return_tensors="pt").to(device)
    translated = model.generate(**batch)
    op_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    output_list.append(op_text)
# assert (
#     tgt_text[0]
#     == "California's largest electricity provider has turned off power to hundreds of thousands of customers."
# )

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
output_list

[["The Palestinian Authority formally becomes the 123rd member of the International Criminal Court.<n>The move gives the court jurisdiction over alleged crimes in Palestinian territories.<n>Palestinians signed the ICC's founding Rome Statute in January."],
 ['Theia, a white-and-black bully breed mix, was apparently hit by a car and buried in a field.<n>Four days later, the dog staggers to a farm and is taken in by a worker.<n>She needs surgery to fix a dislocated jaw and a caved-in sinus cavity.'],
 ["Mohammad Javad Zarif is the Iranian foreign minister.<n>He is the opposite number in talks with the U.S. over Iran's nuclear program.<n>He received a hero's welcome as he arrived in Iran on a sunny Friday morning."],
 ["One of the five had a heart-related issue on Saturday and has been discharged but hasn't left the area.<n>They were exposed to Ebola in Sierra Leone in March, but none developed the deadly virus."],
 ["A student has admitted to hanging a noose from a tree near a student un

In [7]:
from nltk.translate.bleu_score import sentence_bleu
import nltk
from rouge import Rouge
lenlist = []
rougelist = []
bleulist = []
rouge = Rouge()
for i in range(10):
    print("Article ",i)
    print()
    print("Reference Summary: \n",test_dataset['reference_summary'][i])
    print()
    print("Predicted Summary: \n", output_list[i][0])
    print()
    print('SCORES\n')
    reference_summary_tokens = nltk.word_tokenize(test_dataset['reference_summary'][i])
    generated_summary_tokens = nltk.word_tokenize(output_list[i][0])
    bleu_score = sentence_bleu([reference_summary_tokens], generated_summary_tokens)
    print('BLEU: ',bleu_score)
    bleulist.append(bleu_score)
    print()
    rougescore = rouge.get_scores(output_list[i], [test_dataset['reference_summary'][i]])
    print('Rouge: ', rougescore)
    rougelist.append(rougescore)
    print()
    print('Length: ', len(output_list[i][0].split()))
    lenlist.append(len(output_list[i][0].split()))
    print('_______________________________________________________________________________________')
    print()

Article  0

Reference Summary: 
 Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

Predicted Summary: 
 The Palestinian Authority formally becomes the 123rd member of the International Criminal Court.<n>The move gives the court jurisdiction over alleged crimes in Palestinian territories.<n>Palestinians signed the ICC's founding Rome Statute in January.

SCORES

BLEU:  0.08892786873926031

Rouge:  [{'rouge-1': {'r': 0.3, 'p': 0.3103448275862069, 'f': 0.3050847407641483}, 'rouge-2': {'r': 0.18181818181818182, 'p': 0.18181818181818182, 'f': 0.18181817681818196}, 'rouge-l': {'r': 0.3, 'p': 0.3103448275862069, 'f': 0.3050847407641483}}]

Length:  32
_______________________________________________________________________________________

Article  1

Reference Summary: 
 Theia, a bully breed mix, was apparently

C:\Users\saira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\saira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [15]:
rougelist[0]

[{'rouge-1': {'r': 0.3, 'p': 0.3103448275862069, 'f': 0.3050847407641483},
  'rouge-2': {'r': 0.18181818181818182,
   'p': 0.18181818181818182,
   'f': 0.18181817681818196},
  'rouge-l': {'r': 0.3, 'p': 0.3103448275862069, 'f': 0.3050847407641483}}]

In [24]:
finalrouge = { 'rouge-1': {'r': 0, 'p': 0, 'f':0}, 'rouge-2': {'r': 0, 'p': 0, 'f':0}, 'rouge-l': {'r': 0, 'p': 0, 'f':0}}

for i in rougelist:
    finalrouge['rouge-1']['r'] += i[0]['rouge-1']['r']
    finalrouge['rouge-1']['p'] += i[0]['rouge-1']['p']
    finalrouge['rouge-1']['f'] += i[0]['rouge-1']['f']
    finalrouge['rouge-2']['r'] += i[0]['rouge-2']['r']
    finalrouge['rouge-2']['p'] += i[0]['rouge-2']['p']
    finalrouge['rouge-2']['f'] += i[0]['rouge-2']['f']
    finalrouge['rouge-l']['r'] += i[0]['rouge-l']['r']
    finalrouge['rouge-l']['p'] += i[0]['rouge-l']['p']
    finalrouge['rouge-l']['f'] += i[0]['rouge-l']['f']

In [25]:
finalrouge

{'rouge-1': {'r': 2.8571493265610917,
  'p': 2.8880010431525416,
  'f': 2.8228934192944504},
 'rouge-2': {'r': 1.1886717889156913,
  'p': 1.1606702121408004,
  'f': 1.1617027723272608},
 'rouge-l': {'r': 2.670612530024295,
  'p': 2.7231381580520524,
  'f': 2.652222423623455}}

In [26]:
finalrouge['rouge-1']['r'] = finalrouge['rouge-1']['r']/10
finalrouge['rouge-1']['p'] = finalrouge['rouge-1']['p']/10
finalrouge['rouge-1']['f'] = finalrouge['rouge-1']['f']/10
finalrouge['rouge-2']['r'] = finalrouge['rouge-2']['r']/10
finalrouge['rouge-2']['p'] = finalrouge['rouge-2']['p']/10
finalrouge['rouge-2']['f'] = finalrouge['rouge-2']['f']/10
finalrouge['rouge-l']['r'] = finalrouge['rouge-l']['r']/10
finalrouge['rouge-l']['p'] = finalrouge['rouge-l']['p']/10
finalrouge['rouge-l']['f'] = finalrouge['rouge-l']['f']/10

In [27]:
finalrouge

{'rouge-1': {'r': 0.28571493265610914,
  'p': 0.28880010431525416,
  'f': 0.28228934192944505},
 'rouge-2': {'r': 0.11886717889156913,
  'p': 0.11606702121408004,
  'f': 0.11617027723272608},
 'rouge-l': {'r': 0.2670612530024295,
  'p': 0.27231381580520525,
  'f': 0.2652222423623455}}

In [11]:
ableu = sum(bleulist)/len(bleulist)
ableu

0.06262286114440777

In [11]:
avg = sum(lenlist)/len(lenlist)
print(avg)

31.4


In [37]:
lenlist

[250, 240, 213, 195, 177, 216, 249, 149, 168, 144]

In [22]:
print(rouge.get_scores(output_list[0], [test_dataset['reference_summary'][0]]))

[{'rouge-1': {'r': 0.3, 'p': 0.3103448275862069, 'f': 0.3050847407641483}, 'rouge-2': {'r': 0.18181818181818182, 'p': 0.18181818181818182, 'f': 0.18181817681818196}, 'rouge-l': {'r': 0.3, 'p': 0.3103448275862069, 'f': 0.3050847407641483}}]


In [19]:
output_list[0]

["The Palestinian Authority formally becomes the 123rd member of the International Criminal Court.<n>The move gives the court jurisdiction over alleged crimes in Palestinian territories.<n>Palestinians signed the ICC's founding Rome Statute in January."]

In [21]:
test_dataset['reference_summary'][0]

'Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .\nIsrael and the United States opposed the move, which could open the door to war crimes investigations against Israelis .'

In [16]:
test_dataset['article'][0]

'(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony

In [17]:
op_text

['Bob Barker returns to "The Price Is Right" for the first time in eight years.<n>The 91-year-old TV legend hosted the classic "Lucky Seven" game.']

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

src_text = [
    """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

model_name = "google/pegasus-cnn_dailymail"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model.generate(**batch)
op_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

## PEGASUS XSUM

In [1]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

src_text = [
    """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
assert (
    tgt_text[0]
    == "California's largest electricity provider has turned off power to hundreds of thousands of customers."
)

C:\Users\saira\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:

tgt_text[0]

"California's largest electricity provider has turned off power to hundreds of thousands of customers."

In [3]:
teststr = 'बहुत धन्यवाद'

In [4]:
s = teststr.split()

In [5]:
s 

['बहुत', 'धन्यवाद']

In [7]:
import fasttext
import fasttext.util
#fasttext.util.download_model('hi')
ft = fasttext.load_model('wiki.hi/wiki.hi.bin')
word = "नृत्य"
print("Embedding Shape is {}".format(ft.get_word_vector(word).shape))
print("Nearest Neighbors to {} are:".format(word))
ft.get_nearest_neighbors(word) 

Embedding Shape is (300,)
Nearest Neighbors to नृत्य are:


[(0.8913929462432861, 'नृत्य।'),
 (0.8440190553665161, 'नृत्यगान'),
 (0.8374733924865723, 'नृत्यगीत'),
 (0.8336297869682312, 'नृत्यों'),
 (0.8265783190727234, 'नृत्यरत'),
 (0.7971948385238647, 'नृत्यकला'),
 (0.7879464626312256, 'नृत्त'),
 (0.7682990431785583, 'नृतक'),
 (0.7622954845428467, 'नृत्यरचना'),
 (0.7602956295013428, 'नृत्यग्राम')]

In [6]:
type(ft.get_word_vector(s[0]))
#ft.get_nearest_neighbors(s[0])

numpy.ndarray

In [18]:
if s[0] in ft.words:
    print(1)

1


In [ ]:
newlist = "नृत्य"

In [5]:

import numpy as np
def get_average_vec(inp : str):
    inplist = inp.split()
    vecdim = 300
    wordvec = np.zeros(vecdim)
    count=0
    for i in inplist:
        if i in ft.words:
            count+=1
            wordvec += ft.get_word_vector(i)
    if count!=0:
        wordvec = wordvec/count
    else:
        wordvec = wordvec+0.00001
    return wordvec

In [8]:
strr = get_average_vec(teststr)
print(strr)
print()
print(strr.shape)

[-0.02488105  0.12246264 -0.81524038 -0.28542497  0.01536997  0.44094588
  0.10729999  0.09070779  0.26483056  0.25655618  0.1733663   0.01676136
  0.0467561   0.0392098   0.20525776  0.03438711  0.11203054 -0.05129933
 -0.07216304 -0.10814821  0.18925538  0.14774781  0.1941355   0.36458281
  0.28388801 -0.25187763 -0.10543862 -0.12988207  0.32688189  0.34439722
  0.05002906 -0.03717792 -0.01823475  0.02169145 -0.209034   -0.24639808
 -0.03120209  0.26498556 -0.11709271 -0.05766741 -0.18513666 -0.12007378
 -0.08492159 -0.022178    0.02825657 -0.09473052  0.0041271  -0.19831679
  0.01266807 -0.40915851  0.06940969  0.46744771  0.00852609  0.08003618
  0.09500353 -0.07616238  0.02124996 -0.18044493 -0.09044835  0.18222073
  0.13601469  0.01938097  0.27844106 -0.04260344  0.15227334 -0.12661927
  0.12727353  0.13575071  0.33893105 -0.18248206 -0.11807907 -0.06596973
  0.06157661 -0.17421921  0.03566225  0.2912169  -0.07428086 -0.05891106
  0.06447134  0.04850382 -0.02100991  0.03007243 -0

In [3]:

# A = ft.get_word_vector(s[0])
# B = ft.get_word_vector(s[0])

# #cosine = np.dot(A,B)/(norm(A)*norm(B))
# cosine = np.dot(A,B)/(norm(A)*norm(B))
# check = norm(A - B)
# print(cosine)

In [6]:
import numpy as np
from numpy.linalg import norm
def get_cosine_similarity(inputvector1, inputvector2):
    assert (inputvector1.size == 300 and inputvector2.size == 300)
    cosine = np.dot(inputvector1, inputvector2)/(norm(inputvector1)*norm(inputvector2))
    return cosine

In [11]:
A = ft.get_word_vector(s[0])
B = ft.get_word_vector(s[1])
similarity = get_cosine_similarity(A, B)

In [12]:
similarity

0.23954187

301

In [15]:
newlist1 = "नृत्य नृत्यगान"
newlist2 = "नृत्यगीत नृत्यरत"
get_vec_1 = get_average_vec(newlist1)
get_vec_2 = get_average_vec(newlist2)
similarity = get_cosine_similarity(get_vec_1, get_vec_2)
print(similarity)

0.9260828981095699


In [12]:


testingdf = pd.read_csv('test.csv/test.csv')

In [13]:
testingdf.head()

,headline,article
0,"पठानकोट पहुंचे PM मोदी, एयरबेस का जायजा ले बॉर...",प्रधानमंत्री नरेंद्र मोदी पठानकोट एयरबेस पहुंच...
1,सचिन ने देशवासियों को समर्पित किया अपना दोहरा शतक,सचिन तेंदुलकर ने एकदिवसीय अंतरराष्ट्रीय क्रिके...
2,एनआईए करेगी छत्तीसगढ़ में सुरक्षा खामियों की ज...,केंद्रीय गृह राज्य मंत्री आर. पी. एन. सिंह ने ...
3,सीधी बात: शाह बोले- हमारा बस चलता तो अब तक मं...,भारतीय जनता पार्टी (बीजेपी) के राष्ट्रीय अध्यक...
4,"ऋषभ पंत के पास यूनिक टैलेंट, उसके साथ छेड़छाड़ न...",ऋषभ पंत की कभी कभार इस बात के लिए आलोचना की जा...


In [14]:
testingdf['headline'][1]

'सचिन ने देशवासियों को समर्पित किया अपना दोहरा शतक'

In [15]:
ip1 = "प्रधानमंत्री नरेंद्र मोदी ने पठानकोट एयरबेस पहुंचकर सुरक्षा स्थिति की समीक्षा की है और वायुसेना के कर्मियों से मिलकर उनके साथीयों को संबोधित किया है। सुबह करीब 7:30 बजे, प्रधानमंत्री ने पंजाब के पठानकोट का संदर्भ लेकर यात्रा की। उन्होंने एयरबेस की निगरानी के बाद सीमाई क्षेत्रों का हवाई सर्वेक्षण भी करने का निर्णय लिया है। पिछले हफ्ते, पठानकोट एयरबेस पर आतंकियों ने हमला किया था, लेकिन उनकी कोशिश नाकाम रही और सभी 6 पाकिस्तानी आतंकी मार गए थे, जबकि 7 सुरक्षाबलें शहीद हुई थीं। भारत ने दोषियों के खिलाफ कड़ी कार्रवाई की मांग की है । सूचना के अनुसार, प्रधानमंत्री के साथ सेना और एयरफोर्स के चीफ भी मौजूद हो सकते हैं। आतंकियों के हमले के तीसरे दिन, रक्षा मंत्री मनोहर पर्रिकर ने भी पठानकोट का दौरा किया था और उन्होंने साफ तौर पर बताया कि आतंकी विदेश से आए थे और उनके पास पाकिस्तान से आए सामग्री मिली थी। इस मामले की जांच एनआईए को सौंपी गई है । इस बीच, पाकिस्तान के प्रधानमंत्री नवाज शरीफ ने भी भारत के द्वारा पठानकोट आतंकी हमले के संबंध में दी गई सबूतों के आधार पर जांच करने के आदेश दिए हैं। इस पर गुरुवार को उच्च स्तरीय बैठक बुलाई गई, जिसमें पठानकोट हमले पर चर्चा हुई। बैठक के बाद, नवाज शरीफ ने भारत के सबूतों के आधार पर जांच करने के आदेश दिए हैं । सूचना के अनुसार, शरीफ ने भारत की ओर से सौंपे गए सबूतों के आधार पर जांच कराए जाने के लिए सहमति जताई है और इसके बाद उन्होंने इंटेलिजेंस ब्यूरो के चीफ को कार्रवाई करने के लिए सौंप दिया है। बैठक में शरीफ ने भारत के साथ आतंकरोधी नीति के तहत सहयोग बढ़ाने के लिए भी अपनी तैयारी जताई है। इस मौके पर वह अपने राष्ट्रीय सुरक्षा सलाहकार नासिर खान जंजुआ से भारत के एनएसए अजित डोभाल से संपर्क बनाए रखने का आदेश भी दिया है"
ip2 = testingdf['headline'][1]

ip1vec = get_average_vec(ip1)
ip2vec = get_average_vec(ip2)

In [17]:
ip1

'प्रधानमंत्री नरेंद्र मोदी ने पठानकोट एयरबेस पहुंचकर सुरक्षा स्थिति की समीक्षा की है और वायुसेना के कर्मियों से मिलकर उनके साथीयों को संबोधित किया है। सुबह करीब 7:30 बजे, प्रधानमंत्री ने पंजाब के पठानकोट का संदर्भ लेकर यात्रा की। उन्होंने एयरबेस की निगरानी के बाद सीमाई क्षेत्रों का हवाई सर्वेक्षण भी करने का निर्णय लिया है। पिछले हफ्ते, पठानकोट एयरबेस पर आतंकियों ने हमला किया था, लेकिन उनकी कोशिश नाकाम रही और सभी 6 पाकिस्तानी आतंकी मार गए थे, जबकि 7 सुरक्षाबलें शहीद हुई थीं। भारत ने दोषियों के खिलाफ कड़ी कार्रवाई की मांग की है । सूचना के अनुसार, प्रधानमंत्री के साथ सेना और एयरफोर्स के चीफ भी मौजूद हो सकते हैं। आतंकियों के हमले के तीसरे दिन, रक्षा मंत्री मनोहर पर्रिकर ने भी पठानकोट का दौरा किया था और उन्होंने साफ तौर पर बताया कि आतंकी विदेश से आए थे और उनके पास पाकिस्तान से आए सामग्री मिली थी। इस मामले की जांच एनआईए को सौंपी गई है । इस बीच, पाकिस्तान के प्रधानमंत्री नवाज शरीफ ने भी भारत के द्वारा पठानकोट आतंकी हमले के संबंध में दी गई सबूतों के आधार पर जांच करने के आदेश दिए हैं। इस पर गुर

In [20]:
similarity = get_cosine_similarity(ip1vec, ip2vec)
print(similarity)

0.7469793800194328


In [21]:
newdf = pd.read_csv('hindi_summaries.csv')

In [22]:
newdf.head()

,Unnamed: 0,articles,target_summaries,predicted_summaries
0,0,नोट के बदले वोट मामले में आरोपी समाजवादी पार्ट...,समाजवादी पार्टी के पूर्व महासचिव और राज्यसभा स...,नोट के बदले वोट मामले में आरोपी समाजवादी पार्ट...
1,1,"VIDEO: अब सैलरी होगी कम, सेविंग ज़्यादा!",सैलरी के बारे में बात करते वक्‍त पर्सनल बातें ...,"VIDEO: अब सैलरी होगी कम, सेविंग ज़्यादा!!!! VI..."
2,2,रेल नेविगेशन से संबंधित आईआईटी कानपुर की महत्व...,आईआईटी कानपुर द्वारा तैयार सिमरन (सैटेलाइट इमे...,रेल नेविगेशन से संबंधित आईआईटी कानपुर की महत्व...
3,3,बीसीसीआई अध्यक्ष अनुराग ठाकुर ने भविष्य में पा...,भारत किसी भी वक्त पाकिस्तान को तबाह कर सकता है...,बीसीसीआई अध्यक्ष अनुराग ठाकुर ने भविष्य में पा...
4,4,धार्मिक नगरी अयोध्या में दीपावली के अवसर पर शु...,दीपोत्सव 24 से 26 अक्टूबर के बीच अयोध्या में ह...,धार्मिक नगरी अयोध्या में दीपावली के अवसर पर शु...


In [28]:
ipp1 = newdf['target_summaries'][2]
ipp2 = newdf['predicted_summaries'][2]
vec1 = get_average_vec(ipp1)
vec2 = get_average_vec(ipp2)
similarity = get_cosine_similarity(vec1, vec2)
print(similarity)

0.9253260710417014


In [41]:
newdf['articles'][0]

'नोट के बदले वोट मामले में आरोपी समाजवादी पार्टी के पूर्व महासचिव और राज्यसभा सदस्य अमर सिंह को रविवार को दिल्ली के अखिल भारतीय आयुर्विज्ञान संस्थान (एम्स) से छुट्टी मिल गई। अस्पताल से बाहर आते ही सिंह ने कहा कि वह सपा से जुड़े अपने अतीत को हमेशा के लिए भूलना चाहते हैं।  इससे पहले एम्स के गुर्दा रोग विशेषज्ञ डॉक्टर संजय गुप्ता ने बताया कि सिंह फिट हैं और उन्हें छुट्टी दे दी गई है। डॉक्टर गुप्ता ही एम्स में 55 साल के एम्स का उपचार कर रहे थे। सिंह को वोट के बदले नोट मामले में 6 सितंबर को गिरफ्तार कर लिया गया था। स्वास्थ्य आधार पर अदालत ने 24 अक्टूबर को उन्हें जमानत दे दी थी। सिंह को तिहाड़ जेल से 12 सितंबर को एम्स में भर्ती कराया गया था।  सिंगापुर में गुर्दे का प्रत्यारोपण कराने वाले सिंह के गुर्दे में पिछले दिनों कुछ परेशानियां आ गई थीं।  \xa0गुप्ता ने कहा, उनकी हालत अब ठीक है। उनकी सेहत अब पहले से बेहतर है। जब उन्हें यहां लाया गया था तो उनके क्रिएटिनिन की मात्रा में उतार-चढ़ाव आ रहा था। इसके साथ ही उन्हें कुछ अन्य परेशानियां भी थीं।  अस्पताल से बाहर निकलते समय अभिनेत्री और लोकसभा सदस्य

In [40]:
newdf['predicted_summaries'][0]

'नोट के बदले वोट मामले में आरोपी समाजवादी पार्टी के पूर्व महासचिव और राज्यसभा सदस्य अमर सिंह को'

In [42]:
print(len(newdf['predicted_summaries'][0]))
print(len(newdf['predicted_summaries'][1]))
print(len(newdf['predicted_summaries'][2]))
print(len(newdf['predicted_summaries'][3]))


94
59
98
107


In [52]:
with open('newfile', 'w', encoding="utf-8") as file:
    xx = newdf['predicted_summaries'][0]
    file.write(xx)
    file.close

In [47]:
len(b)

59

In [54]:
newdf['predicted_summaries'][0]

'नोट के बदले वोट मामले में आरोपी समाजवादी पार्टी के पूर्व महासचिव और राज्यसभा सदस्य अमर सिंह को'

In [55]:
newdf['articles'][0]

'नोट के बदले वोट मामले में आरोपी समाजवादी पार्टी के पूर्व महासचिव और राज्यसभा सदस्य अमर सिंह को रविवार को दिल्ली के अखिल भारतीय आयुर्विज्ञान संस्थान (एम्स) से छुट्टी मिल गई। अस्पताल से बाहर आते ही सिंह ने कहा कि वह सपा से जुड़े अपने अतीत को हमेशा के लिए भूलना चाहते हैं।  इससे पहले एम्स के गुर्दा रोग विशेषज्ञ डॉक्टर संजय गुप्ता ने बताया कि सिंह फिट हैं और उन्हें छुट्टी दे दी गई है। डॉक्टर गुप्ता ही एम्स में 55 साल के एम्स का उपचार कर रहे थे। सिंह को वोट के बदले नोट मामले में 6 सितंबर को गिरफ्तार कर लिया गया था। स्वास्थ्य आधार पर अदालत ने 24 अक्टूबर को उन्हें जमानत दे दी थी। सिंह को तिहाड़ जेल से 12 सितंबर को एम्स में भर्ती कराया गया था।  सिंगापुर में गुर्दे का प्रत्यारोपण कराने वाले सिंह के गुर्दे में पिछले दिनों कुछ परेशानियां आ गई थीं।  \xa0गुप्ता ने कहा, उनकी हालत अब ठीक है। उनकी सेहत अब पहले से बेहतर है। जब उन्हें यहां लाया गया था तो उनके क्रिएटिनिन की मात्रा में उतार-चढ़ाव आ रहा था। इसके साथ ही उन्हें कुछ अन्य परेशानियां भी थीं।  अस्पताल से बाहर निकलते समय अभिनेत्री और लोकसभा सदस्य

In [14]:
tosummarizedf = pd.read_csv('Hindi_English_Truncated_Corpus.csv/Hindi_English_Truncated_Corpus.csv')

In [15]:
tosummarizedf.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [16]:
data_req = tosummarizedf.loc[tosummarizedf['source']=='tides']

In [17]:
len(data_req)

50000

In [18]:
from datasets import Dataset
data_new = Dataset.from_pandas( data_req )
data_new = data_new.train_test_split(test_size=0.2)

In [19]:
data_new

DatasetDict({
    train: Dataset({
        features: ['source', 'english_sentence', 'hindi_sentence', '__index_level_0__'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['source', 'english_sentence', 'hindi_sentence', '__index_level_0__'],
        num_rows: 10000
    })
})

In [16]:
for val in data_new['test']:
    print(val['english_sentence'])
    break

He was also the ultimate authority who had to pass the final order in a civil case as well as the sentence in a criminal one after taking the opinion of judges on questions of law .


In [18]:
data_new['test']['english_sentence'][0]

'He was also the ultimate authority who had to pass the final order in a civil case as well as the sentence in a criminal one after taking the opinion of judges on questions of law .'

In [21]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

#src_text = test_dataset['article'][0]
output_list = []
model_name = "google/pegasus-cnn_dailymail"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
i = 0
for val in data_new['test']:
    batch = tokenizer(val['english_sentence'], truncation=True, padding="longest", return_tensors="pt").to(device)
    translated = model.generate(**batch)
    op_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    output_list.append(op_text)
    torch.cuda.empty_cache()
    print(i)
    i+=1

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [22]:
output_list

[['People abroad can get advice from a Government representative at the British Embassy, Consulate or High Commission in the country where they live.<n>Find out more at the British Embassy.'],
 ['Her fear of flying has forced her to travel in the cockpits of domestic airliners on several occasions.<n>Her fear of flying has forced her to travel in the cockpits of domestic airliners on several occasions.'],
 ['There was accumulation of stocks at the pitheads to the extent of 14.6 million tonnes by the end of 1976-77.<n>There was accumulation of stocks at the pitheads to the extent of 14.6 million tonnes by the end of 1976-77.'],
 ['This revolution is finished in 30 muhuita, i.e. in onenichthemeion.<n>This revolution is finished in 30 muhuita, i.e. in onenichthemeion.'],
 ['The woman triumphs by her suffering.<n>The woman triumphs by her suffering.<n>The woman triumphs by her suffering.<n>The woman triumphs by her suffering.'],
 ['HSE priced publications are also available from good books

In [27]:
data_new['test']['english_sentence'][4]

'The woman triumphs by her suffering .'

In [28]:
len(output_list)

10000

In [30]:
data_new['test']['english_summarized'] = output_list

TypeError: 'Dataset' object does not support item assignment

In [40]:
def add_col(df):
    return {
        'english_summaries': [[k for k in o] for o in output_list]
    }

In [32]:
finaldata = data_new['test']

In [39]:
for o in output_list:
    for k in o:
        print(k)
    print(o)
    break

People abroad can get advice from a Government representative at the British Embassy, Consulate or High Commission in the country where they live.<n>Find out more at the British Embassy.
['People abroad can get advice from a Government representative at the British Embassy, Consulate or High Commission in the country where they live.<n>Find out more at the British Embassy.']


In [43]:
finald = pd.DataFrame(data_new['test'])

In [45]:
finald['english_summaries'] = output_list

In [48]:
finald.to_csv('english_summarized', encoding="utf-8")

In [49]:
finald

,source,english_sentence,hindi_sentence,__index_level_0__,english_summaries
0,tides,32 . People abroad can get advice from a Gover...,"32 विदेश में व्यक्ति वहाँ ब्रिटीश ऐम्बैसी , का...",91843,[People abroad can get advice from a Governmen...
1,tides,Her fear of flying has forced her to travel in...,ड़ौर के कारण उन्हें घरेलू उड़नों में अक्सर कॉक...,37409,[Her fear of flying has forced her to travel i...
2,tides,There was accumulation of stocks at the pithea...,सन् 1976-77 के अंत तक 1 करोड़ 46 लाख टन तक का ...,113596,[There was accumulation of stocks at the pithe...
3,tides,"This revolution is finished in 30 muhuita , i....",यह परिक्रमा 30 मुहुर्त अर्थात एक नक़्तंदिव में...,12576,"[This revolution is finished in 30 muhuita, i...."
4,tides,The woman triumphs by her suffering .,"स्त्री अपनी यातना में भी , पराजित नहीं होती .",71715,[The woman triumphs by her suffering.<n>The wo...
...,...,...,...,...,...
9995,tides,Besides statements pointing out mistakes or in...,किसी मंत्री द्वारा या अन्य सदस्य द्वारा दिए गए...,11439,[Other items of business which fall under this...
9996,tides,Attempts to play down the loss have begun .,इसके महत्व को भी कम करके आंकने के प्रयास शुरू ...,36997,[Attempts to play down the loss have begun.<n>...
9997,tides,Korean car manufacturer Hyundai though is not ...,मगर कोरियाई कार निर्माता हंड़ई समज्हैते के मुत...,69805,[Korean car manufacturer Hyundai though is not...
9998,tides,He demanded that India should be granted an au...,इसके विपरीत उन्होंने ब्रिटिश प्रभुत्व से पूर्ण...,80155,[He demanded that India should be granted an a...


In [52]:
finale = finald.drop(columns=['__index_level_0__'])

In [51]:
finale.to_csv('english__summarized', encoding="utf-8")

In [4]:
himdisumm = pd.read_csv('hindi_summaries.csv')

In [5]:
himdisumm.head()

,Unnamed: 0,articles,target_summaries,predicted_summaries
0,0,दिल्ली में सुप्रीम कोर्ट के डीज़ल टैक्सियों को...,EXCLUSIVE: दिल्ली में डीजल टैक्सियों पर बैन से...,भारत की राजधानी दिल्ली में चुनावों के लिए टैक्...
1,1,जॉर्डन के ऐतिहासिक दौरे पर पहुंचे राष्ट्रपति प...,जॉर्डन: राष्ट्रपति मुखर्जी ने 86 करोड़ डॉलर के...,भारत के राष्ट्रपति प्रणब मुखर्जी ने जॉर्डन के ...
2,2,पाकिस्तानी नेताओं को विवादित और हास्यास्पद बया...,UN में पाकिस्तान की राजदूत मलीहा लोधी ने कराई ...,पाकिस्तान की राजदूत मलीहा लोधी ने ब्रिटेन के प...
3,3,पीएम नरेंद्र मोदी बायोपिक में विवेक ओबेरॉय ने ...,38 देशों में पीएम नरेंद्र मोदी बायोपिक को रिली...,पीएम नरेंद्र मोदी बायोपिक की रिलीज तारीख 5 अप्...
4,4,"देश, दुनिया, महानगर, खेल, आर्थिक और बॉलीवुड मे...",13 अगस्त 2011: दिनभर की बड़ी खबरें पढ़ें,अन्ना हजारे ने 16 अगस्त से दिल्ली में अनशन शुर...


In [16]:
hindisumip1 = himdisumm['target_summaries'][2]
hindisumip2 = himdisumm['predicted_summaries'][2]
vecc1 = get_average_vec(hindisumip1)
vecc2 = get_average_vec(hindisumip2)
similarity = get_cosine_similarity(vecc1, vecc2)
print(similarity)

0.8331181830711128


In [20]:
len(himdisumm['predicted_summaries'][2].split())

21

In [100]:
import pandas as pd 

sdf = pd.read_csv('translated_sentences(1).csv')
sdf

,Original Hindi Sentences,Translated Sentences
0,['अनुसंधानकर्ताओं को चाहिए कि वे किसी प्रकार क...,"वे इस खतरे को नहीं लेना चाहेंगे, खासकर इसलिए क..."
1,['कुछ प्रकार के शस्त्रक्रिया में शामिल है शरीर...,कुछ तरह के ऑपरेशन में शरीर-LBB-B के एक हिस्से ...
2,['इसके मुख़्य लक्षण तो हैं : तापमान का बढ़ जान...,"इसके मुख्य लक्षण तापमान में उठते हैं, और वुल्फ..."
3,['ऐसी स्थिति में उनका सामुदायिक विकास कार्यक्र...,उसका समुदाय विकास का कार्यक्रम इसलिए स्व-help ...
4,"[""यह बात सर जिओफ्रे दि मोंटमोरंसी नाम के एक मश...",इस तथ्य पर हाल ही में एक ब्रिटिश अधिकारी ने ज़...
...,...,...
9991,"['यह उपन्यास संवाद प्रधान है, इसलिए यह आश्चर्य...",उपन्यास ज़्यादातर संवाद में है और एक आश्चर्य ह...
9992,['एक ही क्षण में बूढ़ा हो गया और बुद्धि का बोझ...,एक पल में मैं बूढ़ा हो गया और मेरी पीठ पर बुद्...
9993,['नमी की उपस्थिति में सल्फर डाईआक्साइड सल्फ्यू...,यह नमी की मौजूदगी में बदल जाती है जो बहुत ही ज...
9994,['इस पृष्ठभूमि में मंदिरों एवं उनमें पूजा व्यव...,मंदिर के निर्माण और संगठित मंदिर की ऐसी पृष्ठभ...


In [4]:
sl = []
for i in sdf['Original Hindi Sentences']:

    sl.append(len(i.split()))


In [6]:
len(sl)

9996

In [8]:
sum(sl)/len(sl)

23.70388155262105

In [1]:
import pandas as pd
hindisumfinal = pd.read_csv('summarized_translated_sentences.csv')

In [2]:
hindisumfinal.count()

Unnamed: 0.1           1700
Unnamed: 0             1700
Original_sentence      1700
translated_sentence    1700
predicted_summaries    1700
dtype: int64

In [3]:
hindisumfinal

,Unnamed: 0.1,Unnamed: 0,Original_sentence,translated_sentence,predicted_summaries
0,0,50,['इंगलैंड का स्वास्थ्य वर्तमान स्थिति'],आज इंग्लैंड का स्वास्थ्य,आज इंग्लैंड में स्वास्थ्य की स्थिति काफी तेज़ी...
1,1,51,['आपने अक़्सर ब्रिटिश साम्राज़्यवाद को बुरा-भल...,लेकिन क्या आपने सोचा है कि यह केवल एक प्रकटन ह...,पाकिस्तान में रविवार को तीन लोगों की मौत हो गई...
2,2,52,['उनके दर्प की यह स्थिति है कि यदि आप उनके साम...,"उनका घमंड ऐसा है कि, यदि आप उन्हें Kuyasson और...",ब्रिटेन के वैज्ञानिक जॉर्ज बुश और उनके सहयोगिय...
3,3,53,['भारतीय सिविल सर्विस के सदसऋ-ऊण्श्छ्ष्-य के र...,"भारतीय नागरिक सेवा के एक सदस्य के रूप में, उसन...",ब्रितानी नागरिक सेवा के एक सदस्य ने अपनी पहचान...
4,4,54,"['इस प्रकार, व्यवहार में भारत में, जैसे कि ग्र...","इस प्रकार, अभ्यास में मौजूद है, जैसे ग्रेविल ऑ...",ग्रेविल ऑस्टिन का अभ्यास भारत के विभिन्न राज्य...
...,...,...,...,...,...
1695,1695,945,['3 अल्प-सूचना प्रश्न : अल्प-सूचना प्रश्न वह प...,संक्षिप्त प्रश्‍न: एक छोटी - सी सूचना प्रश्‍न ...,संक्षिप्त प्रश् न: एक छोटी - सी बात है जो सार्...
1696,1696,946,"['परंतु उसे ज़्यों ही पत्थर पर पछाड़ा गया, वह ...","जैसे ही यह किया गया, उस बच्ची ने बिजली की एक क...",दक्षिण अफ्रीका में एक बच्ची ने आकाश के किनारे ...
1697,1697,947,['वास्तविक निर्माण मोहक रूप से नियमित और लयबद्...,यह सचमुच एक दिलचस्प ढंग से और ताल - मेल से बना...,बीबीसी हिंदी के विशेष कार्यक्रम 'एक मुलाक़ात' ...
1698,1698,948,"['जो आहार हम खाते हैं, उस से हमें जीने और जीवन...",हम भोजन खाते हैं जीवन का आनन्द लेने और जीवन का...,बीबीसी हिंदी के विशेष कार्यक्रम 'एक भोजन' में ...


In [4]:
hindisumfinal.rename(columns={"translated_sentence": "pipeline_1_output", "predicted_summaries": "pipeline_2_output"})

,Unnamed: 0.1,Unnamed: 0,Original_sentence,pipeline_1_output,pipeline_2_output
0,0,50,['इंगलैंड का स्वास्थ्य वर्तमान स्थिति'],आज इंग्लैंड का स्वास्थ्य,आज इंग्लैंड में स्वास्थ्य की स्थिति काफी तेज़ी...
1,1,51,['आपने अक़्सर ब्रिटिश साम्राज़्यवाद को बुरा-भल...,लेकिन क्या आपने सोचा है कि यह केवल एक प्रकटन ह...,पाकिस्तान में रविवार को तीन लोगों की मौत हो गई...
2,2,52,['उनके दर्प की यह स्थिति है कि यदि आप उनके साम...,"उनका घमंड ऐसा है कि, यदि आप उन्हें Kuyasson और...",ब्रिटेन के वैज्ञानिक जॉर्ज बुश और उनके सहयोगिय...
3,3,53,['भारतीय सिविल सर्विस के सदसऋ-ऊण्श्छ्ष्-य के र...,"भारतीय नागरिक सेवा के एक सदस्य के रूप में, उसन...",ब्रितानी नागरिक सेवा के एक सदस्य ने अपनी पहचान...
4,4,54,"['इस प्रकार, व्यवहार में भारत में, जैसे कि ग्र...","इस प्रकार, अभ्यास में मौजूद है, जैसे ग्रेविल ऑ...",ग्रेविल ऑस्टिन का अभ्यास भारत के विभिन्न राज्य...
...,...,...,...,...,...
1695,1695,945,['3 अल्प-सूचना प्रश्न : अल्प-सूचना प्रश्न वह प...,संक्षिप्त प्रश्‍न: एक छोटी - सी सूचना प्रश्‍न ...,संक्षिप्त प्रश् न: एक छोटी - सी बात है जो सार्...
1696,1696,946,"['परंतु उसे ज़्यों ही पत्थर पर पछाड़ा गया, वह ...","जैसे ही यह किया गया, उस बच्ची ने बिजली की एक क...",दक्षिण अफ्रीका में एक बच्ची ने आकाश के किनारे ...
1697,1697,947,['वास्तविक निर्माण मोहक रूप से नियमित और लयबद्...,यह सचमुच एक दिलचस्प ढंग से और ताल - मेल से बना...,बीबीसी हिंदी के विशेष कार्यक्रम 'एक मुलाक़ात' ...
1698,1698,948,"['जो आहार हम खाते हैं, उस से हमें जीने और जीवन...",हम भोजन खाते हैं जीवन का आनन्द लेने और जीवन का...,बीबीसी हिंदी के विशेष कार्यक्रम 'एक भोजन' में ...


In [8]:
cosine_sccores = []
for index, row in hindisumfinal.iterrows():
    # print(row['Original_sentence'].strip("['']"))
    # print(row['predicted_summaries'])
    vec1 = get_average_vec(row['Original_sentence'].strip("['']"))
    vec2 = get_average_vec(row['predicted_summaries'])
    similarity = get_cosine_similarity(vec1, vec2)
    #print(similarity)
    cosine_sccores.append(similarity)

In [12]:
cosine_sccores

[0.7805633802828876,
 0.814935409114611,
 0.8013252395579907,
 0.9078264694336626,
 0.863098178531707,
 0.8238521889496687,
 0.820245469191689,
 0.7441168730383692,
 0.07279296689351847,
 0.8540546861435645,
 0.8395739227755987,
 0.910569617629286,
 0.9052324061335661,
 0.8539218385969146,
 0.7767953019436703,
 0.7529509269231234,
 0.8046172870312127,
 0.7447904160380422,
 0.8674734398326798,
 0.7151757309155721,
 0.8383171693906274,
 0.8475264304028215,
 0.8581606660117953,
 0.785601712981962,
 0.7454259282212236,
 0.8174791884953414,
 0.8305358181742692,
 0.820677565210635,
 0.7697456493806234,
 0.7546390358203346,
 0.8089914021394665,
 0.8486885574627463,
 0.8709800470733934,
 0.8955743407362012,
 0.9176709352352492,
 0.8752116661247298,
 0.8313689218458349,
 0.8177828467372805,
 0.7419780538772468,
 0.9032134146813182,
 0.8885544515969307,
 0.8222574990924512,
 0.7720411771138174,
 0.8236709271645367,
 0.7329322380588608,
 0.8494808136934582,
 0.680736993903828,
 0.9234645715777133

In [9]:
hindisumfinal.head(10)

,Unnamed: 0.1,Unnamed: 0,Original_sentence,translated_sentence,predicted_summaries
0,0,50,['इंगलैंड का स्वास्थ्य वर्तमान स्थिति'],आज इंग्लैंड का स्वास्थ्य,आज इंग्लैंड में स्वास्थ्य की स्थिति काफी तेज़ी...
1,1,51,['आपने अक़्सर ब्रिटिश साम्राज़्यवाद को बुरा-भल...,लेकिन क्या आपने सोचा है कि यह केवल एक प्रकटन ह...,पाकिस्तान में रविवार को तीन लोगों की मौत हो गई...
2,2,52,['उनके दर्प की यह स्थिति है कि यदि आप उनके साम...,"उनका घमंड ऐसा है कि, यदि आप उन्हें Kuyasson और...",ब्रिटेन के वैज्ञानिक जॉर्ज बुश और उनके सहयोगिय...
3,3,53,['भारतीय सिविल सर्विस के सदसऋ-ऊण्श्छ्ष्-य के र...,"भारतीय नागरिक सेवा के एक सदस्य के रूप में, उसन...",ब्रितानी नागरिक सेवा के एक सदस्य ने अपनी पहचान...
4,4,54,"['इस प्रकार, व्यवहार में भारत में, जैसे कि ग्र...","इस प्रकार, अभ्यास में मौजूद है, जैसे ग्रेविल ऑ...",ग्रेविल ऑस्टिन का अभ्यास भारत के विभिन्न राज्य...
5,5,55,"['कुछ सीरियलं ( लिट्ल मिर्ची थोड़ पीपर, सुकन्य...",अपने -LBBB-LBBB के पीछे श्रृंखला की एक स्ट्रिं...,एक बार चूक इंडिया प्रतियोगिता-द-एली की श्रृंखल...
6,6,56,['उदाहरण के लिए तीन प्रकार की घरेलू मक़्खी के ...,"उदाहरण के लिए, यहाँ परोका घरेलू पक्षी हैं जो ह...",दक्षिण अफ़्रीका में क़रीब तीन लाख लोगों की मौत...
7,7,57,['ठोस कचरे के प्रबंधन का वर्तमान तरीका है-शहर ...,मौजूदा प्रबंधन का तरीक़ा है शहर के विभिन्‍न भा...,दक्षिण अफ़्रीका में कोरोना वायरस की वजह से लॉक...
8,8,58,['ठेलेप्होने 0114 279 6600'],टेलिफोन : 0114 2796600,معرفی بهترین مدل موبایل برای استفاده از گوشی -...
9,9,59,['फर्श पर नयी सामग्री बिछाने से पहले सारी पुरा...,उसे अच्छी तरह साफ़ करना चाहिए ।,अफ्रीकी महिलाओं के लिए एक नई चीज़ तैयार होती ह...


In [10]:
hindisumfinal['cosine_scores_pipeline2'] = cosine_sccores

In [11]:
cosine_sccorespipe2 = []
for index, row in hindisumfinal.iterrows():
    # print(row['Original_sentence'].strip("['']"))
    # print(row['predicted_summaries'])
    vec1 = get_average_vec(row['Original_sentence'].strip("['']"))
    vec2 = get_average_vec(row['translated_sentence'])
    similarity = get_cosine_similarity(vec1, vec2)
    #print(similarity)
    cosine_sccorespipe2.append(similarity)

In [13]:
cosine_sccorespipe2

[0.8382631394003676,
 0.9131873322280348,
 0.9497471262527436,
 0.9534448567135478,
 0.8982184685555415,
 0.86473309978611,
 0.8902924182514976,
 0.941382054179957,
 0.090218187024973,
 0.7739953587961886,
 0.9164361802196715,
 0.917425216411975,
 0.9452497152117968,
 0.9113788974577534,
 0.7869309982451616,
 0.7157352886347026,
 0.953151400034127,
 0.7567039059627225,
 0.9304887251999274,
 0.8638476330736411,
 0.9222522122862575,
 0.9120844520382334,
 0.9181934735780097,
 0.870446419132566,
 0.6943577998385451,
 0.9054707363439939,
 0.9153275158293513,
 0.7996027832547032,
 0.8774033439115696,
 0.9308422844613761,
 0.8386120766747278,
 0.9218636674147767,
 0.826212088189859,
 0.9119260044873612,
 0.9359840106215526,
 0.9080847481962807,
 0.9490264541730365,
 0.8816472625781213,
 0.8535074476547196,
 0.9255133840632406,
 0.9488091948525215,
 0.9395857861288303,
 0.8182273561039569,
 0.949583369948552,
 0.7174145466957111,
 0.8788058604837291,
 0.7919916031295414,
 0.9638820958385345,
 

In [14]:
hindisumfinal['cosine_scores_pipeline1'] = cosine_sccorespipe2

In [44]:
hindisumfinal = hindisumfinal.rename(columns={"translated_sentence": "pipeline_1_output", "predicted_summaries": "pipeline_2_output"})

In [65]:
hindisumfinal.head(11)

,Unnamed: 0.1,Unnamed: 0,Original_sentence,pipeline_1_output,pipeline_2_output,cosine_scores_pipeline2,cosine_scores_pipeline1,Results
0,0,50,['इंगलैंड का स्वास्थ्य वर्तमान स्थिति'],आज इंग्लैंड का स्वास्थ्य,आज इंग्लैंड में स्वास्थ्य की स्थिति काफी तेज़ी...,0.780563,0.838263,PIPELINE_1
1,1,51,['आपने अक़्सर ब्रिटिश साम्राज़्यवाद को बुरा-भल...,लेकिन क्या आपने सोचा है कि यह केवल एक प्रकटन ह...,पाकिस्तान में रविवार को तीन लोगों की मौत हो गई...,0.814935,0.913187,PIPELINE_1
2,2,52,['उनके दर्प की यह स्थिति है कि यदि आप उनके साम...,"उनका घमंड ऐसा है कि, यदि आप उन्हें Kuyasson और...",ब्रिटेन के वैज्ञानिक जॉर्ज बुश और उनके सहयोगिय...,0.801325,0.949747,PIPELINE_1
3,3,53,['भारतीय सिविल सर्विस के सदसऋ-ऊण्श्छ्ष्-य के र...,"भारतीय नागरिक सेवा के एक सदस्य के रूप में, उसन...",ब्रितानी नागरिक सेवा के एक सदस्य ने अपनी पहचान...,0.907826,0.953445,PIPELINE_1
4,4,54,"['इस प्रकार, व्यवहार में भारत में, जैसे कि ग्र...","इस प्रकार, अभ्यास में मौजूद है, जैसे ग्रेविल ऑ...",ग्रेविल ऑस्टिन का अभ्यास भारत के विभिन्न राज्य...,0.863098,0.898218,PIPELINE_1
5,5,55,"['कुछ सीरियलं ( लिट्ल मिर्ची थोड़ पीपर, सुकन्य...",अपने -LBBB-LBBB के पीछे श्रृंखला की एक स्ट्रिं...,एक बार चूक इंडिया प्रतियोगिता-द-एली की श्रृंखल...,0.823852,0.864733,PIPELINE_1
6,6,56,['उदाहरण के लिए तीन प्रकार की घरेलू मक़्खी के ...,"उदाहरण के लिए, यहाँ परोका घरेलू पक्षी हैं जो ह...",दक्षिण अफ़्रीका में क़रीब तीन लाख लोगों की मौत...,0.820245,0.890292,PIPELINE_1
7,7,57,['ठोस कचरे के प्रबंधन का वर्तमान तरीका है-शहर ...,मौजूदा प्रबंधन का तरीक़ा है शहर के विभिन्‍न भा...,दक्षिण अफ़्रीका में कोरोना वायरस की वजह से लॉक...,0.744117,0.941382,PIPELINE_1
8,8,58,['ठेलेप्होने 0114 279 6600'],टेलिफोन : 0114 2796600,معرفی بهترین مدل موبایل برای استفاده از گوشی -...,0.072793,0.090218,PIPELINE_1
9,9,59,['फर्श पर नयी सामग्री बिछाने से पहले सारी पुरा...,उसे अच्छी तरह साफ़ करना चाहिए ।,अफ्रीकी महिलाओं के लिए एक नई चीज़ तैयार होती ह...,0.854055,0.773995,PIPELINE_2


In [63]:
cross_cosine_scores = []
for index, row in hindisumfinal.iterrows():
    # print(row['Original_sentence'].strip("['']"))
    # print(row['predicted_summaries'])
    vec1 = get_average_vec(row['pipeline_1_output'])
    vec2 = get_average_vec(row['pipeline_2_output'])
    similarity = get_cosine_similarity(vec1, vec2)
    #print(similarity)
    cross_cosine_scores.append(similarity)

In [64]:
cross_cosine_scores

[0.7773515615098898,
 0.7870969079358875,
 0.8235377820346989,
 0.9435734885455355,
 0.9141283698254526,
 0.9032136595452916,
 0.7595841423082171,
 0.7434888654864507,
 0.15595802282886026,
 0.8324127094024534,
 0.8372532828860894,
 0.9545181118226296,
 0.913834089234913,
 0.8811213900949586,
 0.7881536771202605,
 0.8099086808133134,
 0.8498957597942622,
 0.7708187390727823,
 0.8893283892416168,
 0.8331551037437789,
 0.9164478604527279,
 0.9108564154659508,
 0.8204860524931231,
 0.880612705622382,
 0.8055493631819037,
 0.8540697015204775,
 0.8645663877898665,
 0.7782666927347249,
 0.8068258462004416,
 0.7368622522536766,
 0.8872905058430541,
 0.8687536758813345,
 0.8952314730327448,
 0.9179605974008763,
 0.9364965536881281,
 0.8856891883662653,
 0.845655394602015,
 0.8464654363777094,
 0.7987739521144139,
 0.9589745540415727,
 0.9426911893343561,
 0.852733637284888,
 0.8843489970614196,
 0.7734325142096214,
 0.8156960027928309,
 0.8196437969990329,
 0.6418730576775954,
 0.9445579836877

In [66]:
hindisumfinal['cross_cosine_score'] = cross_cosine_scores

In [67]:
hindisumfinal.head(10)

,Unnamed: 0.1,Unnamed: 0,Original_sentence,pipeline_1_output,pipeline_2_output,cosine_scores_pipeline2,cosine_scores_pipeline1,Results,cross_cosine_score
0,0,50,['इंगलैंड का स्वास्थ्य वर्तमान स्थिति'],आज इंग्लैंड का स्वास्थ्य,आज इंग्लैंड में स्वास्थ्य की स्थिति काफी तेज़ी...,0.780563,0.838263,PIPELINE_1,0.777352
1,1,51,['आपने अक़्सर ब्रिटिश साम्राज़्यवाद को बुरा-भल...,लेकिन क्या आपने सोचा है कि यह केवल एक प्रकटन ह...,पाकिस्तान में रविवार को तीन लोगों की मौत हो गई...,0.814935,0.913187,PIPELINE_1,0.787097
2,2,52,['उनके दर्प की यह स्थिति है कि यदि आप उनके साम...,"उनका घमंड ऐसा है कि, यदि आप उन्हें Kuyasson और...",ब्रिटेन के वैज्ञानिक जॉर्ज बुश और उनके सहयोगिय...,0.801325,0.949747,PIPELINE_1,0.823538
3,3,53,['भारतीय सिविल सर्विस के सदसऋ-ऊण्श्छ्ष्-य के र...,"भारतीय नागरिक सेवा के एक सदस्य के रूप में, उसन...",ब्रितानी नागरिक सेवा के एक सदस्य ने अपनी पहचान...,0.907826,0.953445,PIPELINE_1,0.943573
4,4,54,"['इस प्रकार, व्यवहार में भारत में, जैसे कि ग्र...","इस प्रकार, अभ्यास में मौजूद है, जैसे ग्रेविल ऑ...",ग्रेविल ऑस्टिन का अभ्यास भारत के विभिन्न राज्य...,0.863098,0.898218,PIPELINE_1,0.914128
5,5,55,"['कुछ सीरियलं ( लिट्ल मिर्ची थोड़ पीपर, सुकन्य...",अपने -LBBB-LBBB के पीछे श्रृंखला की एक स्ट्रिं...,एक बार चूक इंडिया प्रतियोगिता-द-एली की श्रृंखल...,0.823852,0.864733,PIPELINE_1,0.903214
6,6,56,['उदाहरण के लिए तीन प्रकार की घरेलू मक़्खी के ...,"उदाहरण के लिए, यहाँ परोका घरेलू पक्षी हैं जो ह...",दक्षिण अफ़्रीका में क़रीब तीन लाख लोगों की मौत...,0.820245,0.890292,PIPELINE_1,0.759584
7,7,57,['ठोस कचरे के प्रबंधन का वर्तमान तरीका है-शहर ...,मौजूदा प्रबंधन का तरीक़ा है शहर के विभिन्‍न भा...,दक्षिण अफ़्रीका में कोरोना वायरस की वजह से लॉक...,0.744117,0.941382,PIPELINE_1,0.743489
8,8,58,['ठेलेप्होने 0114 279 6600'],टेलिफोन : 0114 2796600,معرفی بهترین مدل موبایل برای استفاده از گوشی -...,0.072793,0.090218,PIPELINE_1,0.155958
9,9,59,['फर्श पर नयी सामग्री बिछाने से पहले सारी पुरा...,उसे अच्छी तरह साफ़ करना चाहिए ।,अफ्रीकी महिलाओं के लिए एक नई चीज़ तैयार होती ह...,0.854055,0.773995,PIPELINE_2,0.832413


In [46]:
result_list = []
pipe1 = 0
pipe2 = 0
for index, row in hindisumfinal.iterrows():
    if row['cosine_scores_pipeline1']>row['cosine_scores_pipeline2']:
        result_list.append('PIPELINE_1')
        pipe1+=1
    elif row['cosine_scores_pipeline1']<row['cosine_scores_pipeline2']:
        result_list.append('PIPELINE_2')
        pipe2+=1
    elif row['cosine_scores_pipeline1']==row['cosine_scores_pipeline2']:
        result_list.append('SAME SCORE')

In [47]:
hindisumfinal['Results'] = result_list

In [68]:
hindisumfinal.head(10)

,Unnamed: 0.1,Unnamed: 0,Original_sentence,pipeline_1_output,pipeline_2_output,cosine_scores_pipeline2,cosine_scores_pipeline1,Results,cross_cosine_score
0,0,50,['इंगलैंड का स्वास्थ्य वर्तमान स्थिति'],आज इंग्लैंड का स्वास्थ्य,आज इंग्लैंड में स्वास्थ्य की स्थिति काफी तेज़ी...,0.780563,0.838263,PIPELINE_1,0.777352
1,1,51,['आपने अक़्सर ब्रिटिश साम्राज़्यवाद को बुरा-भल...,लेकिन क्या आपने सोचा है कि यह केवल एक प्रकटन ह...,पाकिस्तान में रविवार को तीन लोगों की मौत हो गई...,0.814935,0.913187,PIPELINE_1,0.787097
2,2,52,['उनके दर्प की यह स्थिति है कि यदि आप उनके साम...,"उनका घमंड ऐसा है कि, यदि आप उन्हें Kuyasson और...",ब्रिटेन के वैज्ञानिक जॉर्ज बुश और उनके सहयोगिय...,0.801325,0.949747,PIPELINE_1,0.823538
3,3,53,['भारतीय सिविल सर्विस के सदसऋ-ऊण्श्छ्ष्-य के र...,"भारतीय नागरिक सेवा के एक सदस्य के रूप में, उसन...",ब्रितानी नागरिक सेवा के एक सदस्य ने अपनी पहचान...,0.907826,0.953445,PIPELINE_1,0.943573
4,4,54,"['इस प्रकार, व्यवहार में भारत में, जैसे कि ग्र...","इस प्रकार, अभ्यास में मौजूद है, जैसे ग्रेविल ऑ...",ग्रेविल ऑस्टिन का अभ्यास भारत के विभिन्न राज्य...,0.863098,0.898218,PIPELINE_1,0.914128
5,5,55,"['कुछ सीरियलं ( लिट्ल मिर्ची थोड़ पीपर, सुकन्य...",अपने -LBBB-LBBB के पीछे श्रृंखला की एक स्ट्रिं...,एक बार चूक इंडिया प्रतियोगिता-द-एली की श्रृंखल...,0.823852,0.864733,PIPELINE_1,0.903214
6,6,56,['उदाहरण के लिए तीन प्रकार की घरेलू मक़्खी के ...,"उदाहरण के लिए, यहाँ परोका घरेलू पक्षी हैं जो ह...",दक्षिण अफ़्रीका में क़रीब तीन लाख लोगों की मौत...,0.820245,0.890292,PIPELINE_1,0.759584
7,7,57,['ठोस कचरे के प्रबंधन का वर्तमान तरीका है-शहर ...,मौजूदा प्रबंधन का तरीक़ा है शहर के विभिन्‍न भा...,दक्षिण अफ़्रीका में कोरोना वायरस की वजह से लॉक...,0.744117,0.941382,PIPELINE_1,0.743489
8,8,58,['ठेलेप्होने 0114 279 6600'],टेलिफोन : 0114 2796600,معرفی بهترین مدل موبایل برای استفاده از گوشی -...,0.072793,0.090218,PIPELINE_1,0.155958
9,9,59,['फर्श पर नयी सामग्री बिछाने से पहले सारी पुरा...,उसे अच्छी तरह साफ़ करना चाहिए ।,अफ्रीकी महिलाओं के लिए एक नई चीज़ तैयार होती ह...,0.854055,0.773995,PIPELINE_2,0.832413


In [69]:
pipe1_average = hindisumfinal.loc[:, 'cosine_scores_pipeline1'].mean()
pipe2_average = hindisumfinal.loc[:, 'cosine_scores_pipeline2'].mean()
cross_average = hindisumfinal.loc[:, 'cross_cosine_score'].mean()
print("Pipeline 1 Score:", pipe1)
print("Pipeline 2 Score:",pipe2)
print("Pipeline 1 Average Cosine Score:", pipe1_average)
print("Pipeline 2 Average Cosine Score:", pipe2_average)
print("Cross Pipeline Cosine Score Average:", cross_average)

Pipeline 1 Score: 1518
Pipeline 2 Score: 182
Pipeline 1 Average Cosine Score: 0.8735735059775221
Pipeline 2 Average Cosine Score: 0.8146656576850562
Cross Pipeline Cosine Score Average: 0.8516348714934966


In [70]:
hindisumfinal.head(10)

,Unnamed: 0.1,Unnamed: 0,Original_sentence,pipeline_1_output,pipeline_2_output,cosine_scores_pipeline2,cosine_scores_pipeline1,Results,cross_cosine_score
0,0,50,['इंगलैंड का स्वास्थ्य वर्तमान स्थिति'],आज इंग्लैंड का स्वास्थ्य,आज इंग्लैंड में स्वास्थ्य की स्थिति काफी तेज़ी...,0.780563,0.838263,PIPELINE_1,0.777352
1,1,51,['आपने अक़्सर ब्रिटिश साम्राज़्यवाद को बुरा-भल...,लेकिन क्या आपने सोचा है कि यह केवल एक प्रकटन ह...,पाकिस्तान में रविवार को तीन लोगों की मौत हो गई...,0.814935,0.913187,PIPELINE_1,0.787097
2,2,52,['उनके दर्प की यह स्थिति है कि यदि आप उनके साम...,"उनका घमंड ऐसा है कि, यदि आप उन्हें Kuyasson और...",ब्रिटेन के वैज्ञानिक जॉर्ज बुश और उनके सहयोगिय...,0.801325,0.949747,PIPELINE_1,0.823538
3,3,53,['भारतीय सिविल सर्विस के सदसऋ-ऊण्श्छ्ष्-य के र...,"भारतीय नागरिक सेवा के एक सदस्य के रूप में, उसन...",ब्रितानी नागरिक सेवा के एक सदस्य ने अपनी पहचान...,0.907826,0.953445,PIPELINE_1,0.943573
4,4,54,"['इस प्रकार, व्यवहार में भारत में, जैसे कि ग्र...","इस प्रकार, अभ्यास में मौजूद है, जैसे ग्रेविल ऑ...",ग्रेविल ऑस्टिन का अभ्यास भारत के विभिन्न राज्य...,0.863098,0.898218,PIPELINE_1,0.914128
5,5,55,"['कुछ सीरियलं ( लिट्ल मिर्ची थोड़ पीपर, सुकन्य...",अपने -LBBB-LBBB के पीछे श्रृंखला की एक स्ट्रिं...,एक बार चूक इंडिया प्रतियोगिता-द-एली की श्रृंखल...,0.823852,0.864733,PIPELINE_1,0.903214
6,6,56,['उदाहरण के लिए तीन प्रकार की घरेलू मक़्खी के ...,"उदाहरण के लिए, यहाँ परोका घरेलू पक्षी हैं जो ह...",दक्षिण अफ़्रीका में क़रीब तीन लाख लोगों की मौत...,0.820245,0.890292,PIPELINE_1,0.759584
7,7,57,['ठोस कचरे के प्रबंधन का वर्तमान तरीका है-शहर ...,मौजूदा प्रबंधन का तरीक़ा है शहर के विभिन्‍न भा...,दक्षिण अफ़्रीका में कोरोना वायरस की वजह से लॉक...,0.744117,0.941382,PIPELINE_1,0.743489
8,8,58,['ठेलेप्होने 0114 279 6600'],टेलिफोन : 0114 2796600,معرفی بهترین مدل موبایل برای استفاده از گوشی -...,0.072793,0.090218,PIPELINE_1,0.155958
9,9,59,['फर्श पर नयी सामग्री बिछाने से पहले सारी पुरा...,उसे अच्छी तरह साफ़ करना चाहिए ।,अफ्रीकी महिलाओं के लिए एक नई चीज़ तैयार होती ह...,0.854055,0.773995,PIPELINE_2,0.832413


In [61]:
print(hindisumfinal['Original_sentence'][12].strip("['']"))
print(hindisumfinal['pipeline_1_output'][12])
print(hindisumfinal['pipeline_2_output'][12])

सरकार ने इस उमीद में कि सौदे को लेकर बातचीत फिर शुरू होगी, अभी जांच का कार्यक्षेत्र और उसकी अध्यक्षता के लिए किसी न्यायाधीश की नियुइक्त नहीं की है.
सरकार अब तक संदर्भ की शर्तों को ठीक करने या उस पैनल के सिर पर एक न्यायाधीश नियुक्‍त करने के लिए है जब यह उम्मीद कर रहा है कि सौदा फिर से स्थापित किया जाएगा.
ब्रिटेन के गृह मंत्रालय का कहना है कि वह विवादास्पद क़ानून सौदा को स्थगित करने की कोशिश कर रहा है.


In [ ]:
hindisumfinal.to_csv('FINAL_RESULTS.csv', encoding="utf-8")